In [50]:
import joblib
import json
import pandas as pd
import numpy as np

In [51]:
path = "../data/verify.json"
with open(path) as f:
    data = json.load(f)

In [52]:
df = pd.json_normalize(data)
df.head()

,browser,os,locale,gender,location,sites,time,date
0,Firefox,Ubuntu,ru-RU,m,Singapore/Singapore,"[{'site': 'mail.google.com', 'length': 63}, {'...",13:06:00,2019-04-22
1,Chrome,Windows 10,ru-RU,m,Singapore/Singapore,"[{'site': 'mail.google.com', 'length': 122}, {...",14:13:00,2019-04-24
2,Firefox,Ubuntu,ru-RU,m,Singapore/Singapore,"[{'site': 'mail.google.com', 'length': 174}, {...",14:01:00,2019-04-26
3,Firefox,Ubuntu,ru-RU,m,Singapore/Singapore,"[{'site': 'mail.google.com', 'length': 473}, {...",14:49:00,2019-04-28
4,Firefox,Windows 10,ru-RU,m,Singapore/Singapore,"[{'site': 'mail.google.com', 'length': 119}, {...",05:55:00,2019-04-29


In [53]:
sites = df["sites"]

In [54]:
cv = joblib.load("../model/countvec.joblib")

In [55]:
sites_enc = cv.transform(sites)

In [56]:
sites_enc = pd.DataFrame(sites_enc.toarray(), columns=cv.get_feature_names())

In [57]:
df.drop("sites", axis = 1, inplace=True)

In [58]:
df["software_language"] = df.locale.apply(lambda x:x[:2])
df['software_country'] = df.locale.apply(lambda x:x[-2:])
df.drop("locale", axis =1 , inplace=True)

In [59]:
df["country"] = df.location.apply(lambda x:x.split('/')[0])
df["city"] = df.location.apply(lambda x:x.split('/')[1])
df.drop("location", axis = 1, inplace=True)

In [60]:
df["hour"] = df.time.apply(lambda x:int(x[:2]))

In [61]:
df.head()

,browser,os,gender,time,date,software_language,software_country,country,city,hour
0,Firefox,Ubuntu,m,13:06:00,2019-04-22,ru,RU,Singapore,Singapore,13
1,Chrome,Windows 10,m,14:13:00,2019-04-24,ru,RU,Singapore,Singapore,14
2,Firefox,Ubuntu,m,14:01:00,2019-04-26,ru,RU,Singapore,Singapore,14
3,Firefox,Ubuntu,m,14:49:00,2019-04-28,ru,RU,Singapore,Singapore,14
4,Firefox,Windows 10,m,05:55:00,2019-04-29,ru,RU,Singapore,Singapore,5


In [62]:
drops = ["time", "date", 'city']
df.drop(drops, axis = 1, inplace=True)

In [63]:
df["russia_software"] = np.where(df.software_country=='RU', "RU", "Other")

df.drop(['software_country',"software_language"] , axis = 1, inplace=True)

In [64]:
sites_enc.columns = sites_enc.columns.str.replace(".", "_")

/var/folders/lj/w4hy005d5vx9qy3lrwz0xdl80000gn/T/ipykernel_39813/161791293.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  sites_enc.columns = sites_enc.columns.str.replace(".", "_")


In [65]:
df = pd.concat([df, sites_enc], axis = 1)

In [76]:
site_cols = sites_enc.columns

In [66]:
feat_eng_pipe = joblib.load("../model/feature_engineer_pipeline.pkl")

In [74]:
pred_data = feat_eng_pipe.transform(df)

/Users/georgemcintire/projects/job_assignments/toptal/George-McIntire-2/assignment_env/lib/python3.9/site-packages/sklearn/compose/_column_transformer.py:427: FutureWarning: Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.
  warnings.warn("Given feature/column names or counts do not match "


In [69]:
cat_cols = ['browser', 'os', 'gender', 'country', 'hour', "russia_software"]

In [70]:
cat_col_names = feat_eng_pipe.named_transformers_["cat"]["ohe"].get_feature_names(cat_cols).tolist()

In [78]:
column_names = cat_col_names + site_cols.tolist()

In [79]:


pred_data = pd.DataFrame(data=pred_data, columns=column_names)


In [81]:
pred_data.head()

,browser_Firefox,browser_Internet Explorer,browser_Safari,os_MacOS,os_Ubuntu,os_Windows 10,os_Windows 7,os_Windows 8,gender_m,country_Brazil,...,yupptv_io,ziffstatic_com,zmedia_vn,zomatobook_com,zoomg_ir,zoozoosexporn_com,zradio_org,zscalergov_net,zubuchygri_com,zuuks_com
0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,-0.003584,-0.004739,-0.006483,-0.004851,-0.004078,-0.003795,-0.006528,-0.005339,-0.003942,-0.004998
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,-0.003584,-0.004739,-0.006483,-0.004851,-0.004078,-0.003795,-0.006528,-0.005339,-0.003942,-0.004998
2,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,-0.003584,-0.004739,-0.006483,-0.004851,-0.004078,-0.003795,-0.006528,-0.005339,-0.003942,-0.004998
3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,-0.003584,-0.004739,-0.006483,-0.004851,-0.004078,-0.003795,-0.006528,-0.005339,-0.003942,-0.004998
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,-0.003584,-0.004739,-0.006483,-0.004851,-0.004078,-0.003795,-0.006528,-0.005339,-0.003942,-0.004998


In [82]:
pred_data.drop("browser_Safari", axis = 1, inplace=True)

In [83]:
select = joblib.load("../model/feature_selector.pkl")

In [84]:
pred_data = select.transform(pred_data)

In [85]:
pred_data.shape

(22700, 83)

In [86]:
model = joblib.load("../model/Random_Forest_model.pkl")

In [88]:
preds = model.predict(pred_data)

In [91]:
pd.value_counts(preds, normalize=True)

0    0.989515
1    0.010485
dtype: float64

In [92]:
df.head()

,browser,os,gender,country,hour,russia_software,0x_tf,139_com,22_cn,247wallst_com,...,yupptv_io,ziffstatic_com,zmedia_vn,zomatobook_com,zoomg_ir,zoozoosexporn_com,zradio_org,zscalergov_net,zubuchygri_com,zuuks_com
0,Firefox,Ubuntu,m,Singapore,13,RU,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Chrome,Windows 10,m,Singapore,14,RU,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Firefox,Ubuntu,m,Singapore,14,RU,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Firefox,Ubuntu,m,Singapore,14,RU,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Firefox,Windows 10,m,Singapore,5,RU,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [97]:
preds_df = pd.DataFrame(index=df.index, data = preds)

In [98]:
preds_df.to_csv("../data/result.csv")